In [24]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

$P_{Fusion}(E,T) \propto \dfrac{1}{(kT)^{3/2}}\ \exp\left(-\dfrac{E}{kT}\right)\ \exp\left(-\dfrac{b}{\sqrt{E}}\right)$  
\
$T_C \approx 1.6\times10^7\ \left(\dfrac{R_0}{R}\right)\left(\dfrac{M}{M_0}\right)$

In [25]:
sun_radius = 6.957*10**8 #metres
sun_mass = 1.989*10**30 #kg
boltzmann = 1.380649*10**-23 #J/K
b = 10**-6

In [26]:
def central_temperature(star_radius, star_mass):

    return 1.6*10**7 * (sun_radius/star_radius) * (star_mass/sun_mass)

In [27]:
def fusion_probability(temperature, energy):
    T = temperature
    k = boltzmann

    return (1/((k*T)**(3/2))) * np.exp(-energy/(k*T)) * np.exp(-b/energy**(1/2))

In [28]:
def plot(figure, figures=1):
    figure.update_layout(
        autosize=True,
        width=(1920/2) * figures,
        height=1080/2,
    )
    figure.show()

Function for finding Gamow peak areas, using numerical integartion by approximating the curve as a series of trapezoids

In [29]:
def get_area(x, y):

    x_step = x[1]-x[0] 
    return np.sum((y[:-1] + y[1:]) * 0.5 * x_step)

Few examples to check function works as intended:  

1. Integrating $x^2$ from 0 to 3: should yield ~9
2. Integrating $2x^3$ from 0 to 2: should yield ~8
3. Integrating $4x + 81$ from 1 to 4: should yield ~273

In [30]:
x_1 = np.linspace(0, 3, 101)
y_1 = x_1**2
print(get_area(x_1, y_1))

x_2 = np.linspace(0, 2, 101)
y_2 = 2*x_2**3
print(get_area(x_2, y_2))

x_3 = np.linspace(1, 4, 101)
y_3 = 4*x_3 + 81
print(get_area(x_3, y_3))

9.00045
8.0008
273.0000000000003


All checks passed

Useful shorthands for initial three plots:

In [31]:
energy = np.array(np.linspace(1*10**-16, 100*10**-16, 1001))
kT = boltzmann*central_temperature(sun_radius, sun_mass)

Plotting "first" exponential term $\exp\left(-\dfrac{E}{kT}\right)$

In [32]:
fig = go.Figure()

exp_1 = np.exp(-energy/kT)

fig.add_trace(go.Scatter(
        x=energy,
        y=exp_1,
        mode='markers',
        name=f""
))

fig.update_layout(
        xaxis_title="Energy [J]",
        yaxis_title="",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)

Plotting "second" exponential term $\exp\left(-\dfrac{b}{\sqrt{E}}\right)$

In [33]:
fig = go.Figure()

exp_2 = np.exp(-b/energy**(1/2))

fig.add_trace(go.Scatter(
        x=energy,
        y=exp_2,
        mode='markers',
        name=f""
))

fig.update_layout(
        xaxis_title="Energy [J]",
        yaxis_title="",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)

Plotting probability of fusion (Gamow peak)

In [34]:
fig = go.Figure()

sun_temperature = central_temperature(sun_radius, sun_mass)
sun_fp = fusion_probability(sun_temperature, energy)
sun_area = get_area(energy, sun_fp)
    
fig.add_trace(go.Scatter(
        x=energy,
        y=sun_fp,
        mode='markers',
        name=f""
))

fig.update_layout(
        xaxis_title="Energy [J]",
        yaxis_title="Fusion Probability",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)
print(f"Sun central temperature Tc ≈ {sun_temperature:.3e} K")
print(f"Gamow-peak area at Tc(sun) ≈ {sun_area:.6e}")

Sun central temperature Tc ≈ 1.600e+07 K
Gamow-peak area at Tc(sun) ≈ 1.191678e-05


Why does Gamow peak appear?

Using known stars (https://en.wikipedia.org/wiki/Main_sequence), I am going to find a relation between star radius and mass.  
Testing a power law relation of the form $y=x^N$ by plotting $log(y)$ againt $log(x)$, where the gradient will give N.

Mass range justification?

In [35]:
stars = ["O2", "O6", "B0", "B5", "A0", "A5", "F0", "F5", "G0", "G2", "G5", "K0", "K5", "M0", "M5", "M8", "L1"]
stellar_radii = np.array([12, 9.8, 7.4, 3.8, 2.5, 1.7, 1.3, 1.2, 1.05, 1, 0.93, 0.85, 0.74, 0.51, 0.18, 0.11, 0.09]) # in solar units
stellar_masses = np.array([100, 35, 18, 6.5, 3.2, 2.1, 1.7, 1.3, 1.10, 1, 0.93, 0.78, 0.69, 0.60, 0.15, 0.08, 0.07]) # in solar units

In [36]:
fig = go.Figure()

x=np.log(stellar_masses)
y=np.log(stellar_radii)

fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name=f""
))

# Fit: log(radius) = N*log(mass) + log(k)
(coefs, cov) = np.polyfit(x, y, 1, cov=True)
N, logk = coefs
N_err = np.sqrt(cov[0,0])
logk_err = np.sqrt(cov[1,1])
k = np.exp(logk)
k_err = k * logk_err

# For a nice straight line to overlay:
x_line = np.linspace(x.min(), x.max(), 200)
y_line = N*x_line + logk

# Add to your existing figure (which currently plots logs).
fig.add_trace(go.Scatter(
    x=x_line,
    y=y_line,
    mode='lines',
    name=f"Fit: R ≈ {k:.2f} M^{N:.2f}"
))

fig.update_layout(
        xaxis_title="log(Stellar Mass)",
        yaxis_title="log(Stellar Radius)",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)
print(f"N = {N:.3f} ± {N_err:.3f},  k = {k:.10f} ± {k_err:.3f}")

N = 0.709 ± 0.031,  k = 0.8499067866 ± 0.052


This finds that $\dfrac{R}{R_0}=k\dfrac{M}{M_0}^{0.71}$, so:  

$T_C \approx 1.6\times10^7\ \left(\dfrac{R_0}{R}\right)\left(\dfrac{M}{M_0}\right)$  

$T_C \approx 1.6\times10^7\ \left(\dfrac{R_0}{kR_0\dfrac{M}{M_0}^{0.71}}\right)\left(\dfrac{M}{M_0}\right)$  

$T_C \approx 1.6\times10^7\ \dfrac{1}{k}\left(\dfrac{M}{M_0}\right)^{-0.71}\left(\dfrac{M}{M_0}\right)$  

$T_C \approx 1.6\times10^7\ \dfrac{1}{k}\left(\dfrac{M}{M_0}\right)^{0.29}$  

$T_C \approx 1.88235\times10^7\ \left(\dfrac{M}{M_0}\right)^{0.29}$ for the fitted $k=0.85$  

Now that we have central temperature as a function of mass, $T_C(M/M_0)$, we can use stellar masses from earlier to find a range of $T_C$ to compute various Gamow peak areas (proportional to fusion rate)

In [37]:
central_temperatures = 1.88235*10**7 * stellar_masses**0.29
areas = [get_area(energy, fusion_probability(temp, energy)) for temp in central_temperatures]


Plotting area against central temperature

In [38]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=central_temperatures,
    y=areas,
    mode='lines+markers',
    name=f"Fit: R ≈ {k:.2f} M^{N:.2f}"
))

fig.update_layout(
        xaxis_title="Central Temperature [K]",
        yaxis_title="Gamow Peak Area",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)

Plotting area against mass

In [39]:
fig = go.Figure()

x=np.log(stellar_masses)
y=np.log(areas)

fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name=f""
))

# Fit: log(radius) = N*log(mass) + log(k)
(coefs, cov) = np.polyfit(x, y, 1, cov=True)
N, logk = coefs
N_err = np.sqrt(cov[0,0])
logk_err = np.sqrt(cov[1,1])
k = np.exp(logk)
k_err = k * logk_err

# For a nice straight line to overlay:
x_line = np.linspace(x.min(), x.max(), 200)
y_line = N*x_line + logk

# Add to your existing figure (which currently plots logs).
fig.add_trace(go.Scatter(
    x=x_line,
    y=y_line,
    mode='lines',
    name=f"Fit: R ≈ {k:.2f} M^{N:.2f}"
))

fig.update_layout(
        xaxis_title="log(M/M_0)",
        yaxis_title="log(area)",
        legend=dict(x=0.65, y=0.98)
)

plot(fig)
print(f"N = {N:.3f} ± {N_err:.3f},  k = {k:.10f} ± {k_err:.3f}")

N = 2.479 ± 0.084,  k = 0.0000363312 ± 0.000


**Final scaling and interpretation**  

Using the mass–radius fit $\dfrac{R}{R_0} \approx k\dfrac{M}{M_0}^N,$ where my fit finds $N = 0.71$ and $k = 0.85$, the central temperature relation was found to be:  

$$
T_C(M) \approx 1.6\times10^7\ \dfrac{1}{k}\left(\dfrac{M}{M_0}\right)^{1-N}
$$  
$$
T_C(M) \approx 1.88235\times10^7\ \left(\dfrac{M}{M_0}\right)^{0.29}
$$
  
Evaluating the Gamow-peak area (fusion reaction rate) at these $T_C(M)$ values and fitting for a power law finds:   

$$
\text{Rate} \propto  M^{N_{rate}},
$$
where $N_{rate} = 2.5$. Assuming the available fuel will be proportional to the mass, M, the main-sequence lifetime scales as:  

$$
\text{Lifetime}(M) \propto \dfrac{M}{\text{Rate}} \propto \dfrac{M}{M^{2.5}}
$$
$$
\text{Lifetime}(M) \propto M^{-1.5}
$$

Explaining final result:

1. What does it mean for the dependence of lifetime on mass for stars?  

(For main sequence)
- Larger mass = higher fusion reaction rate (more fuel to burn)
- Higher reaction rate = Faster reactions = shorter lifetime
- Temperature also scales up with mass, so a higher mass means higher temperature, means higher particle speed, higher reaction rate
2. 
3. How valid is my result? Limitations? Why does it differ from given model: https://rainman.astro.illinois.edu/ddr/stellar/intermediate.html